# 2. Ligand-Receptor Inference

In [ ]:
# prep data in R;
# see details in '/PHShome/jn22/kidney/LIANA_cell2cell/Python_notebooks/convert_kidneySeurat_to_anndata.ipynb'


library(Seurat)
# library('hdf5r')
# library(cli)
# library('digest')
# library(htmltools)
# library('later')
# library(purrr)
# library('httpuv')
# library(tidyr)
library(SeuratDisk)
library(magrittr)
library(dplyr)
library(data.table)
library(ggplot2)


read.csv(file = '../kidney/test_coda/kid_celltype.csv')->kid_celltype# the csv prepared by '/PHShome/jn22/kidney/prep_input_to_tascCODA.R'
head(kid_celltype)

readRDS('../kidney/kidney.Rds')->kid
kid[[]] %>% View()

if (identical(kid_celltype$cellId,rownames(kid[[]]))){
  kid_celltype[,-1] %>%
    tibble::column_to_rownames(var = 'cellId')->kid@meta.data
}else{
  '!!!please double check'
}

SaveH5Seurat(kid, filename = "kidney.h5Seurat")
Convert("kidney.h5Seurat", dest = "h5ad")
unlink("kidney.h5Seurat")

## Environment Setup and Directories
#### switch back to LIANA env

In [ ]:
#switch back to LIANA env
import os
import numpy as np
import pandas as pd
import scipy

import scanpy as sc

import liana as li
import plotnine as p9

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

data_path = '.'
output_folder = os.path.join(data_path, 'kidney_liana_output_for_cell2cell/')
os.mkdir(output_folder)

## Data Reminder

In [ ]:
adata = sc.read_h5ad(os.path.join(data_path, 'kidney.h5ad'))

just as a quick reminder, let's visualize the cell types and samples in the data.

In [ ]:
adata.obs.head()

In [ ]:
np.sort(adata.obs.celltype_level2a.unique())

In [ ]:
np.sort(adata.obs.replace(to_replace={"celltype_level2a": "M-MDSC"}, 
                  value="MDSC").celltype_level2a.unique())

In [ ]:
adata.obs=adata.obs.replace(to_replace={"celltype_level2a": "M-MDSC"}, 
                  value="MDSC")

In [ ]:
np.sort(adata.obs.replace(to_replace={"celltype_level2a": "distal_tubule_epithelial cells"}, 
                  value={"celltype_level2a": "distal_tubule_epithelial_cells"}).celltype_level2a.unique())

In [ ]:
adata.obs=adata.obs.replace(to_replace={"celltype_level2a": "distal_tubule_epithelial cells"}, 
                  value={"celltype_level2a": "distal_tubule_epithelial_cells"})

In [ ]:
adata.obs.head()

In [ ]:
np.sort(adata.obs.celltype_level2a.unique())

In [ ]:
adata.obs["laipingCluster"] = adata.obs["laipingCluster"].apply(str)
# adata.obs.laipingCluster=adata.obs.laipingCluster.astype('category')

adata.obs.celltype_level2a=adata.obs.celltype_level2a.apply(str)
adata.obs.celltype_level2a=adata.obs.celltype_level2a.astype('category')

adata.obs.info()

In [ ]:
# plot pre-annotated cell types
sc.pl.umap(adata, color=['laipingCluster','celltype_level2a']) # would do 'sample_new' only if we integrate

In [ ]:
li.method.show_methods()

In [ ]:
[i for i in li.method.show_methods()['Reference']]

### Score Distributions

In [ ]:
adata.var

In [ ]:
#sc.pp.highly_variable_genes(adata, flavor = 'seurat', n_top_genes = 5000, batch_key="sample")

In [ ]:
# pick a sample to infer the communication scores for
ctr_adata = adata[adata.obs['sample']=='CTR'].copy()
d7_adata = adata[adata.obs['sample']=='D7'].copy()
m4_adata = adata[adata.obs['sample']=='M4'].copy()

In [ ]:
if input("Ture to load already existed h5ad, and skip 'li.method.rank_aggregate' and saving h5ad"):
    sadata = sc.read_h5ad(os.path.join(data_path, 'kidney_CTR.h5ad'))
else:
    print('Have not load h5ad, the below cell will asign')

In [ ]:
%%time

sadata=d7_adata.copy()
li.method.rank_aggregate(sadata, 
                           # groupby='laipingCluster', 
                           groupby='celltype_level2a', 
                           resource_name = 'mouseconsensus',
                           expr_prop=0.1, # must be expressed in expr_prop fraction of cells
                           min_cells = 5,
                           n_perms = 1000, 
                           use_raw = False, # run on log- and library-normalized counts
                         # use_raw = True,
                           verbose = True, 
                           inplace = True,
                         #target_organism = 10090
                          )
#check and save h5ad
sadata

sadata.write_h5ad(os.path.join(data_path, ''.join(['kidney_', whichSample,'.h5ad'])))

LIANA's results are by default save to the `liana_res` slot in `adata.uns`

In [ ]:
liana_res = sadata.uns['liana_res'].copy()
# only keep those that are not liana's ranks
liana_res.head()

In [ ]:
liana_res[liana_res['source'].isin(["MDSC", "cancer_cell_c11", "cancer_cell_c5"]) & liana_res.target.isin(["MDSC", "cancer_cell_c11", "cancer_cell_c5"])].head()
# liana_res.source.isin(["MDSC", "cancer_cell_c11", "cancer_cell_c5"]) | liana_res.target.isin(["MDSC", "cancer_cell_c11", "cancer_cell_c5"])

In [ ]:
liana_res[liana_res.source.isin(["cancer_cell_c11", "cancer_cell_c5"]) & liana_res.target.isin(["MDSC"])]

In [ ]:
liana_res[liana_res.source.isin(["MDSC"]) & liana_res.target.isin(["cancer_cell_c11", "cancer_cell_c5"])]

In [ ]:
li.method.show_methods()

In [ ]:
# convert to long format by index, and each score and value in different columns
liana_res = liana_res.loc[:, liana_res.columns[~liana_res.columns.str.contains(pat = 'rank')]]
liana_res = liana_res.melt(id_vars=['source', 'target', 'ligand_complex', 'receptor_complex'], var_name='score', value_name='value')

liana_res['score'] = liana_res['score'].astype('category')

In [ ]:
%%time 
(p9.ggplot(liana_res, p9.aes(x='value', fill='score')) + 
 p9.geom_density(alpha=0.5) + 
 p9.facet_wrap('~score', scales='free') +
 p9.theme_bw() +
 p9.theme(figure_size=(8, 8))
 )

### Single-Sample Dotplot

In [ ]:
msk1 = np.isin(sadata.uns['liana_res']["source"], np.array(["5", "11", "0", "2", "8", "13"]))
msk2 = np.isin(sadata.uns['liana_res']["target"], np.array(['0', '2', "8", "13","5", "11"]))

sadata.uns['liana_res'][msk1 & msk2]

# msk[1:10]

In [ ]:
del msk1
del msk2
sadata.uns['liana_res'].info()

In [ ]:
sadata2 = sadata.copy()
sadata2.uns['liana_res']["target"] = sadata2.uns['liana_res']["target"].apply(str)
sadata2.uns['liana_res']["source"] = sadata2.uns['liana_res']["source"].apply(str)
sadata2.uns['liana_res']['source']= sadata2.uns['liana_res']['source'].astype('category')

In [ ]:
msk1 = np.isin(sadata2.uns['liana_res']["source"], np.array(["5", "11", "0", "2", "8", "13"]))
msk2 = np.isin(sadata2.uns['liana_res']["target"], np.array(['0', '2', "8", "13","5", "11"]))

sadata2.uns['liana_res'][msk1 & msk2]

In [ ]:
%%time
li.pl.dotplot(
    adata=sadata2,
    # liana_res=sadata2.uns['liana_res'],
    colour='lr_means',
    size='cellphone_pvals',
    source_labels=["MDSC", "cancer_cell_c11", "cancer_cell_c5"],
    # source_labels=["cancer_cell_c11", "cancer_cell_c5"],
    target_labels=["MDSC", "cancer_cell_c11", "cancer_cell_c5"],
    
    # colour="magnitude_rank",
    # size="specificity_rank",
    
    inverse_colour=True,  # we inverse sign since we want small p-values to have large sizes
    inverse_size=True,
    # We choose only the cell types which we wish to plot
    # source_labels=["B", "pDC", "Macrophages"],
    # target_labels=["T", "Mast", "pDC", "NK"],


    # since the rank_aggregate can also be interpreted as a probability distribution
    # we can again filter them according to their specificity significance
    # yet here the interactions are filtered according to
    # how consistently highly-ranked is their specificity across the methods
    # filterby="specificity_rank",
    # filter_lambda=lambda x: x <= 0.05,
   
    # again, we can also further order according to magnitude
    orderby="lr_means",
    orderby_ascending=True,  # prioritize those with lowest values
    top_n=20,  # and we want to keep only the top 20 interactions
    figure_size=(9, 5),
    size_range=(1, 6),
)

In [ ]:
adata.uns['liana_res']['receptor_complex'].unique()

In [ ]:
# adata.uns['liana_res'].to_csv("kindey_all_liana_results.csv")
adata.uns['liana_res'].to_csv("kindey_all_celltype_level2a_liana_results.csv")

In [ ]:
import re

# np.sort(adata.uns['liana_res']['receptor_complex'].unique())
receptor_complex_of_interest=[]
for i in adata.uns['liana_res']['receptor_complex'].unique():
    if re.search(".*csf.*", i, re.I):
        receptor_complex_of_interest.append(re.search(".*csf.*", i, re.I).group())
receptor_complex_of_interest

# receptor_complex_of_interest=[re.search(".*csf.*", i, re.I).group() for i in adata.uns['liana_res']['receptor_complex'].unique() if re.search(".*csf.*", i, re.I)]

In [ ]:
# t_pattern="|".join('Ccl2,Ccl4,Ccl5,Ccl7,Ccl8,Ccl12,Cxcl3,Cxcl14,Ccl3,Ccl5,Ccl7,Cxcl1,Cxcl2,Cxcl5'.split(','))
t_pattern=('Ccl2,Ccl4,Ccl5,Ccl7,Ccl8,Ccl12,Cxcl3,Cxcl14,Ccl3,Ccl5,Ccl7,Cxcl1,Cxcl2,Cxcl5'.replace(',','|'))
import re
ligand_complex_of_interest=list(set([re.search(t_pattern, i, re.I).group() for i in adata.uns['liana_res']['ligand_complex'].unique() if re.search(t_pattern, i, re.I)]))
ligand_complex_of_interest

We can also generate a dotplot for the most highly-ranked ligand-receptor interactions for each sample.
Let's pick the first two distinct interaction in the list, and see how they look like in the `dotplot_by_sample`.

In [ ]:
adata2 = adata.copy()
adata2.uns['liana_res']["target"] = adata2.uns['liana_res']["target"].apply(str)
adata2.uns['liana_res']["source"] = adata2.uns['liana_res']["source"].apply(str)

In [ ]:
adata2

In [ ]:
%%time
li.pl.dotplot_by_sample(
    adata=adata2,
    colour='lr_means',
    size='cellphone_pvals',
    source_labels=["MDSC", "cancer_cell_c11", "cancer_cell_c5"],
    # target_labels=['0', '2', "8", "13","5", "11"],
    target_labels=["MDSC", "cancer_cell_c11", "cancer_cell_c5"],
    ligand_complex = ligand_complex_of_interest,
    # receptor_complex= ['CD3D', 'KLRD1'],
    # receptor_complex= receptor_complex_of_interest,
    sample_key='sample',
    inverse_colour=False,
    inverse_size=True,
    figure_size=(9, 27),
    size_range=(1, 6),
)